In [81]:
# -*- coding: utf-8 -*-
from nltk.tag.sequential import UnigramTagger
from nltk.corpus import brown
from nltk.tokenize import TreebankWordTokenizer
from nltk import tokenize
import numpy as np
from gensim.models import word2vec
import keras
import keras.preprocessing.sequence
from keras.models import Sequential
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import SimpleRNN, LSTM, GRU
from keras.layers.core import Dense, Dropout
from keras.layers.wrappers import TimeDistributed
from keras.layers import Convolution1D, Bidirectional
from keras.models import load_model
import keras.optimizers as optimizers


In [90]:
allCats=brown.categories()
numSamples=10000 #Change this
allsents=brown.tagged_sents(categories=allCats)[0:numSamples]
allsentsTest=brown.tagged_sents(categories=allCats)[10000:11000]
allsentsCleaned=[]
allsentsTestCleaned=[]
#Clean the labels to get rid of the TL
for i in range(0,len(allsents)):
    newSent=[]
    curSent=allsents[i]
    for word in curSent:
        newSent.append((word[0],word[1].split("-")[0]))
    allsentsCleaned.append(newSent)
for i in range(0,len(allsentsTest)):
    newSent=[]
    curSent=allsentsTest[i]
    for word in curSent:
        newSent.append((word[0],word[1].split("-")[0]))
    allsentsTestCleaned.append(newSent)
allLabels=[]
allWords=[]
for sent in allsents:
    for word in sent:
        allWords.append(word[0].lower())
        allLabels.append(word[1].split("-")[0])
uniqueWords=np.unique(allWords)
uniqueLabels=np.unique(allLabels)
print("num seqs:")
print(len(allsents))
print("num labeled words:")
print(len(allWords))



num seqs:
10000
num labeled words:
219770


In [133]:
trainedNetworkDir='./model/'
modelName='testModel.h5'
allLens=[]
for sent in allsentsCleaned:
    allLens.append(len(sent))
print("Max sequence len :"+str(np.max(allLens)))

allLens=[]
for sent in allsentsTest:
    allLens.append(len(sent))
def makeW2Vmodel(inputSents):
    tokenizedSents=[]
    for i in range(0,len(inputSents)):
        curSent=inputSents[i]
        curTokenizedSent=[]
        for word in curSent:
            curTokenizedSent.append(word[0].lower())
        tokenizedSents.append(curTokenizedSent)
    #Now that we have the training strings, we need to make the word embeddings
    w2vModel=word2vec.Word2Vec(tokenizedSents, iter=200, sg=0)
    vocab=list(w2vModel.wv.wv.vocab.keys())
    return w2vModel, vocab
def convertToEmbedInts(inputSents, vocab, maxLen):
    embedInts=[]
    for i in range(0,len(inputSents)):
        curSent=inputSents[i]
        embedding=[]
        for word in curSent:
            curWord=word[0].lower()
            if curWord in vocab:
                embedding.append(vocab.index(curWord)+1)
            else:
                embedding.append(0)
        embedInts.append(keras.preprocessing.sequence.pad_sequences([embedding], maxlen=maxLen)[0])
    return np.asarray(embedInts)
def convertToBinaryLabels(inputSents, possLabels, maxLen):
    binaryLabels=[]
    for i in range(0,len(inputSents)):
        curBinaryLabel=np.zeros((maxLen,len(possLabels)))
        #Set default label to be 0 for all indicies
        curBinaryLabel[:,0]=1
        curSent=inputSents[i]
        for j in range(0,len(curSent)):
            #Fill in from the end
            curLabel=curSent[-1*j][1]
            if curLabel in possLabels:
                curInd=list(possLabels).index(curLabel)
                curBinaryLabel[-1*j,curInd]=1
                curBinaryLabel[-1*j,0]=0
        binaryLabels.append(curBinaryLabel)
    return np.asarray(binaryLabels)

def makeEmbeddingMatrix(model, vocab):
    mykeys=vocab
    embeddingMatrix=np.zeros((len(mykeys)+1,model.vector_size))
    ind=1 #First entry is all zeros
    for key in mykeys:
        embeddingMatrix[ind]=model[key]
        ind=ind+1
    return embeddingMatrix



Max sequence len :141


In [134]:
maxLen=150
curModel, curVocab=makeW2Vmodel(allsentsCleaned)
embedMat=makeEmbeddingMatrix(curModel,curVocab)
myints=convertToEmbedInts(allsents,curVocab,maxLen)
mylabels=convertToBinaryLabels(allsents,uniqueLabels, maxLen)

/home/becky/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:58: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [135]:
from keras.callbacks import ModelCheckpoint
import keras.losses as losses
model = Sequential()
model.add(Embedding(np.shape(embedMat)[0], np.shape(embedMat)[1],input_length=maxLen,weights=[embedMat],
                    trainable=True))
model.add(LSTM(256,return_sequences=True))
#model.add(LSTM(256, return_sequences=True))
#model.add(TimeDistributed(Dense(128, activation='tanh')))
model.add(TimeDistributed(Dense(len(uniqueLabels), activation='relu')))#Make this adapt to ds
model.add(Dense(len(uniqueLabels),activation='sigmoid'))
# model.compile('RMSprop', 'categorical_crossentropy', metrics=['accuracy'])
adam = optimizers.Adam(lr=0.001)
model.compile(loss ='categorical_crossentropy', metrics=['accuracy'],optimizer=adam)
numTrain=8000
x_train=myints[0:numTrain,:]
y_train=mylabels[0:numTrain,:,:]
x_val=myints[numTrain:,:]
y_val=mylabels[numTrain:,:,:]
checkpointer = ModelCheckpoint(filepath=trainedNetworkDir+modelName, verbose=1, save_best_only=True)
# model=load_model('./test.h5')

In [150]:
y_train[5][0]

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.])

In [ ]:
# model.compile('RMSprop', 'categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train, batch_size=128, epochs=200, verbose=1, validation_data=(x_val,y_val), callbacks=[checkpointer])

Train on 8000 samples, validate on 2000 samples
Epoch 1/200
8000/8000 [==============================] - 18s 2ms/step - loss: 2.4378 - acc: 0.4930 - val_loss: 1.2885 - val_acc: 0.8422
Epoch 2/200
8000/8000 [==============================] - 16s 2ms/step - loss: 1.0280 - acc: 0.8667 - val_loss: 0.6768 - val_acc: 0.8422
Epoch 3/200
8000/8000 [==============================] - 16s 2ms/step - loss: 0.3793 - acc: 0.8677 - val_loss: 0.3117 - val_acc: 0.8910
Epoch 4/200
8000/8000 [==============================] - 16s 2ms/step - loss: 0.1821 - acc: 0.9494 - val_loss: 0.1785 - val_acc: 0.9512
Epoch 5/200
8000/8000 [==============================] - 16s 2ms/step - loss: 0.1361 - acc: 0.9622 - val_loss: 0.1566 - val_acc: 0.9562
Epoch 6/200
8000/8000 [==============================] - 16s 2ms/step - loss: 0.1190 - acc: 0.9660 - val_loss: 0.1428 - val_acc: 0.9594
Epoch 7/200
8000/8000 [==============================] - 17s 2ms/step - loss: 0.1079 - acc: 0.9685 - val_loss: 0.1339 - val_acc: 0.9611


In [126]:
# model = load_model('./model/testModel.h5')
myintsTest=convertToEmbedInts(allsentsTest,curVocab,maxLen)
predictions=model.predict(myintsTest)

In [113]:
myintsTest.shape

(1000, 150)

In [58]:
def convertPredictionsToLabels(inputSents,inputPredictions, possLabels):
    #Need input sents to get the correct length
    outLabs=[]
    for i in range(0,len(inputSents)):
        curLen=len(inputSents[i])
        curPred=predictions[i,-1*curLen:,:]
        curSentLabs=[]
        for j in range(0, curLen):
            curSentLabs.append(possLabels[np.argmax(curPred[j,:])])
#             print(np.argmax(curPred[j,:]))
        outLabs.append(curSentLabs)
    return outLabs


In [127]:
# myintsTest.shape
print(uniqueLabels)
print(convertPredictionsToLabels(allsentsTest, predictions, uniqueLabels)
)

['' "'" "''" '(' ')' '*' ',' '.' ':' 'ABL' 'ABN' 'ABX' 'AP' 'AP$' 'AT' 'BE'
 'BED' 'BED*' 'BEDZ' 'BEDZ*' 'BEG' 'BEM' 'BEN' 'BER' 'BER*' 'BEZ' 'BEZ*'
 'CC' 'CD' 'CD$' 'CS' 'DO' 'DO*' 'DOD' 'DOD*' 'DOZ' 'DOZ*' 'DT' 'DT$'
 'DT+BEZ' 'DTI' 'DTS' 'DTX' 'EX' 'EX+BEZ' 'FW' 'HV' 'HV*' 'HVD' 'HVD*'
 'HVG' 'HVN' 'HVZ' 'HVZ*' 'IN' 'JJ' 'JJR' 'JJS' 'JJT' 'MD' 'MD*' 'MD+HV'
 'NIL' 'NN' 'NN$' 'NNS' 'NNS$' 'NP' 'NP$' 'NP+BEZ' 'NPS' 'NPS$' 'NR' 'NR$'
 'NRS' 'OD' 'PN' 'PN$' 'PN+HVZ' 'PP$' 'PP$$' 'PPL' 'PPLS' 'PPO' 'PPS'
 'PPS+BEZ' 'PPS+HVZ' 'PPS+MD' 'PPSS' 'PPSS+BEM' 'PPSS+BER' 'PPSS+HV'
 'PPSS+HVD' 'PPSS+MD' 'QL' 'QLP' 'RB' 'RB$' 'RB+BEZ' 'RBR' 'RBT' 'RN' 'RP'
 'TO' 'UH' 'VB' 'VB+PPO' 'VBD' 'VBG' 'VBN' 'VBZ' 'WDT' 'WDT+BEZ' 'WP$'
 'WPO' 'WPS' 'WPS+BEZ' 'WQL' 'WRB' 'WRB+BEZ' '``']
[['AT', 'AT', 'AT', 'AT', 'AT', 'AT', 'AT', 'AT', 'AT', 'AT', 'AT', 'AT', 'AT', 'AT', 'AT', 'AT', 'AT', 'AT', 'AT', 'AT', 'AT', 'AT', 'AT', 'AT', 'AT', 'AT', 'AT', 'AT', 'AT', 'AT', 'AT', 'AT', 'AT', 'AT', 'AT', 'AT', 'AT', 'A

In [33]:
a = 0
for i in range(len(allsentsTest)):
    a = a+len(allsentsTest[i])

print(a)

2271


In [57]:
uniqueLabels

array(['', "'", "''", '(', ')', '*', ',', '.', ':', 'ABL', 'ABN', 'ABX',
       'AP', 'AT', 'BE', 'BED', 'BEDZ', 'BEDZ*', 'BEG', 'BEM', 'BEN',
       'BER', 'BEZ', 'CC', 'CD', 'CS', 'DO', 'DO*', 'DOD', 'DOD*', 'DOZ',
       'DOZ*', 'DT', 'DT$', 'DTI', 'DTS', 'DTX', 'EX', 'FW', 'HV', 'HVD',
       'HVG', 'HVN', 'HVZ', 'IN', 'JJ', 'JJR', 'JJS', 'JJT', 'MD', 'MD*',
       'NN', 'NN$', 'NNS', 'NNS$', 'NP', 'NP$', 'NPS', 'NPS$', 'NR', 'NR$',
       'OD', 'PN', 'PP$', 'PP$$', 'PPL', 'PPLS', 'PPO', 'PPS', 'PPS+BEZ',
       'PPS+HVZ', 'PPSS', 'PPSS+BEM', 'PPSS+BER', 'PPSS+HVD', 'PPSS+MD',
       'QL', 'QLP', 'RB', 'RBR', 'RBT', 'RP', 'TO', 'UH', 'VB', 'VBD',
       'VBG', 'VBN', 'VBZ', 'WDT', 'WP$', 'WPO', 'WPS', 'WRB', '``'],
      dtype='<U8')